# Warehouse Problem - Lagrangian relaxation

Let us recall the original MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c}  & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

$\mathcal{L}(x,y,\lambda) = \sum_w c_wx_w + \sum_{w,c} t_{w,c}y_{w,c} + \sum_c \lambda_c (1 - \sum_w y_{w,c})$

In [1]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 2 are relaxed.

#### 2. Implement the lagrangian relaxation.

In [29]:
from docplex.mp.model import Model

def lagrangian_relaxation(rlambda=[1]*nbStores):
    model = Model(name='lagrangian')
    
     # Decision variables
    x = {w: model.binary_var(name=f'x_{w}') for w in Warehouses}  # 1 if warehouse w is open
    y = {(w, s): model.binary_var(name=f'y_{w}_{s}') for w in Warehouses for s in Stores}  # 1 if customer s is served by warehouse w
    
    # Lagrangian objective function
    Lagrangian_obj = sum(fixed * x[w] for w in Warehouses) + \
                     sum(supplyCost[s][w] * y[w, s] for w in Warehouses for s in Stores) + \
                     sum(rlambda[s] * (1 - sum(y[w, s] for w in Warehouses)) for s in Stores)

    # Constraints: if a warehouse is not open, it cannot supply any store
    for w in range(nbWarehouses):
        for s in range(nbStores):
            model.add_constraint(y[w, s] <= x[w], f'supply_{s}_{w}')  # If warehouse w is closed, it cannot serve any store

    # Constraints: each warehouse cannot serve more stores than its capacity
    for w in range(nbWarehouses):
        model.add_constraint(model.sum(y[w, s] for s in range(nbStores)) <= capacity[w] * x[w], f'capacity_{w}')  # Respect warehouse capacity

    
    # Set the objective
    model.minimize(Lagrangian_obj)

    # Solve the model
    solution = model.solve()
    
    # Print the solution
    if solution:
        print(f"Lagrangian Objective Value: {solution.objective_value:.2f}.")
      
    else:
        print("No feasible solution found for the Lagrangian Objective function.")
    
    relaxation_constraints = [1 - sum(y[w, s].solution_value for w in Warehouses) for s in Stores]
    
    x_sol = [x[w].solution_value for w in Warehouses]
    y_sol = {(w, s): y[w,s].solution_value for w in Warehouses for s in Stores}
    
    return (model.objective_value,  # the objective value
            solution, # solution
            relaxation_constraints,
            x_sol,
           y_sol)  # slack of the relaxed constraints

#### 3. Implement the Cutting Plane Method

1. State the master problem
2. Keep track of the best feasible solution
3. Use the solution to the master problem to solve the Lagrangian subproblem

In [31]:
rlambda = [fixed]*nbStores
fsol = None
iteration = 0
step = 1
UB = M*nbStores
bestFeasibleValue = UB
bestFeasible = 0


mdl = Model(name='lagrangian')
u = mdl.continuous_var_list(nbStores, name='lambda', ub=M)
v = mdl.continuous_var(name='lagrangian_dual')
mdl.maximize(v)
iteration = 0 

while True:

    print("-"*40)
    print(f"Iteration # {iteration+1}")
    sub_obj, msol, relaxation_constraints, x_sol, y_sol = lagrangian_relaxation(rlambda)
    
    mdl.add_constraint(v <= (sum(fixed * x_sol[w] for w in Warehouses) + \
                             sum(supplyCost[s][w] * y_sol[w, s] for w in Warehouses for s in Stores)) + \
                               sum(u[s]*relaxation_constraints[s] for s in Stores)
                      )

    solution = mdl.solve()

    rlambda = [u[s].solution_value for s in Stores]

    master_obj = solution.objective_value

    

    iteration += 1
    print(f"Duality gap = {master_obj - obj}\n")
    print(f"Master Objective Value {master_obj:.2f}")

    tolerance = 10e-5
    # Check if all relaxation constraints are satisfied
    if abs(master_obj - sub_obj) <= tolerance:
        break
        
  
print('========================')
print('Lagrangian dual: ' + str(UB))
if bestFeasible != 0:
    print('-- best feasible --')
    print(bestFeasible)

----------------------------------------
Iteration # 1
Lagrangian Objective Value: 282.00.
Duality gap = 1380.0

Master Objective Value 1662.00
----------------------------------------
Iteration # 2
Lagrangian Objective Value: -1103.00.
Duality gap = 958.3333333333333

Master Objective Value 1240.33
----------------------------------------
Iteration # 3
Lagrangian Objective Value: -535.67.
Duality gap = 662.3333333333333

Master Objective Value 944.33
----------------------------------------
Iteration # 4
Lagrangian Objective Value: -787.67.
Duality gap = 581.0

Master Objective Value 863.00
----------------------------------------
Iteration # 5
Lagrangian Objective Value: -1111.33.
Duality gap = 445.625

Master Objective Value 727.62
----------------------------------------
Iteration # 6
Lagrangian Objective Value: -1052.12.
Duality gap = 361.3571428571429

Master Objective Value 643.36
----------------------------------------
Iteration # 7
Lagrangian Objective Value: -994.05.
Duality